In [ ]:
!pip install wordcloud

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi
from wordcloud import WordCloud

In [ ]:
# --- 시각화 설정 ---
# 한글 폰트 설정 (Windows: Malgun Gothic, Mac: AppleGothic)
import platform
if platform.system() == 'Darwin': # Mac
    plt.rc('font', family='AppleGothic')
    font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf' # 워드클라우드용
else: # Windows
    plt.rc('font', family='Malgun Gothic')
    font_path = 'c:/Windows/Fonts/malgun.ttf' # 워드클라우드용

plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 크롤링된 파일명 입력
# 예: reviews_1562700_200_{app_id}.json
# reviews_381210_200_Dead_by_Daylight.json
filename_input = input("분석할 JSON 파일명을 입력하세요: ")
# 테스트를 위해 파일명을 직접 지정하거나 위 input 주석을 해제하세요.
# filename_input = "reviews_Example_Game"
if not filename_input.lower().endswith('.json'):
    filename = filename_input + '.json'
else:
    filename = filename_input

base_path = "../dataSet"
full_path = os.path.join(base_path, filename)
absolute_path = os.path.abspath(full_path)
print(f"로드할 파일 경로: {full_path}")

In [ ]:
try:
    with open(absolute_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # JSON 구조에 따라 데이터 로드 방식 변경 (리뷰 리스트가 'reviews' 키 아래에 있다고 가정)
    reviews_data = data.get('reviews', data) if isinstance(data, dict) else data
    df = pd.DataFrame(reviews_data)
    
    # 게임 이름 추출 (파일명을 기준으로)
    game_name = filename.split('_')[3].replace('.json', '')

    # 전처리: 텍스트 없는 행 제거
    df = df.dropna(subset=['review_text'])
    df['playtime_hours'] = df['playtime_forever'] / 60
    
    print(f"✅ '{game_name}' 데이터 로드 완료. 총 리뷰 수: {len(df)}개")
    print("\n--- 데이터 미리보기 ---")
    print(df.head())

except FileNotFoundError:
    print(f"❌ 오류: 파일을 찾을 수 없습니다: {full_path}. 경로를 확인해 주세요.")
    df = pd.DataFrame()
except Exception as e:
    print(f"❌ 데이터 로드 중 오류 발생: {e}")
    df = pd.DataFrame()

In [ ]:
output_dir = os.path.abspath(base_path)

safe_game_name = game_name.replace(' ', '_').replace(':', '_') # 파일명에 부적합한 문자 대체
GAME_OUTPUT_DIR = os.path.join(output_dir, safe_game_name)

# 디렉토리가 없으면 생성합니다.
os.makedirs(GAME_OUTPUT_DIR, exist_ok=True)
print(f"✅ 시각화 결과 저장 디렉토리: {GAME_OUTPUT_DIR} 생성 완료.")

In [ ]:
# 성향별 키워드 사전 정의
KEYWORDS = {
    "Narrative (서사)": ["스토리", "서사", "감동", "엔딩", "캐릭터", "몰입", "대사", "연출", "배경", "더빙"],
    "Freedom (자유도)": ["자유도", "오픈월드", "탐험", "상호작용", "선택", "커스터마이징", "비선형", "루트"],
    "Stability (안정성)": ["최적화", "버그", "프레임", "렉", "튕김", "서버", "운영", "잔렉", "불안정", "발열"],
    "Challenge (도전)": ["난이도", "컨트롤", "보스", "피지컬", "패턴", "도전", "소울", "어려움", "노력", "하드"]
}

In [ ]:
def calculate_persona_vector(review_text):
    """단일 리뷰 텍스트를 분석하여 성향 벡터 (4축 점수)를 반환합니다."""
    scores = {k: 0 for k in KEYWORDS.keys()}
    total_hits = 0
    
    if pd.isna(review_text):
        return {k: 0.0 for k in KEYWORDS.keys()}

    # 텍스트 내 키워드 카운팅
    for category, words in KEYWORDS.items():
        for word in words:
            # 정규 표현식을 사용하여 해당 단어의 출현 횟수 카운트
            count = len(re.findall(word, review_text, re.IGNORECASE))
            scores[category] += count
            total_hits += count
            
    # 정규화 (전체 합이 1이 되도록 비중 계산)
    if total_hits == 0:
        return {k: 0.0 for k in KEYWORDS.keys()}
    
    persona_vector = {k: round(v / total_hits, 3) for k, v in scores.items()}
    return persona_vector

In [ ]:
# 각 리뷰 텍스트에 함수 적용하여 'persona_vector' 컬럼 생성
if not df.empty:
    df['persona_vector'] = df['review_text'].apply(calculate_persona_vector)

    # 벡터 딕셔너리를 개별 컬럼으로 분리
    vector_df = df['persona_vector'].apply(pd.Series)
    vector_df.columns = ['S_' + col for col in vector_df.columns]

    # 중복 컬럼방지
    existing_cols = [col for col in vector_df.columns if col in df.columns]
    if existing_cols:
        df = df.drop(columns=existing_cols)
        
    df = pd.concat([df, vector_df], axis=1)

    # 'Dominant_Persona' (가장 높은 성향) 컬럼 추가 -> 시각화 색상 구분용
    df['Dominant_Persona'] = vector_df.idxmax(axis=1)
    
    print("\n✅ 성향 벡터 계산 완료.")
    display(df[['review_text', 'Dominant_Persona'] + list(vector_df.columns)].head(3))
else:
    print("분석할 DataFrame이 비어 있어 성향 벡터 계산을 건너뜁니다.")

In [ ]:
if not df.empty:
    # 1. 평균 성향 벡터 계산
    persona_cols = [col for col in df.columns if col.startswith('S_')]
    means = df[persona_cols].mean().values.flatten().tolist()
    categories = [col.replace('S_', '') for col in persona_cols]

    # 2. 레이더 차트 설정
    N = len(categories)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1] # 폐곡선을 위해 첫 번째 값 반복
    means += means[:1]

    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, polar=True)

    # 3. 축 그리기
    plt.xticks(angles[:-1], categories, color='grey', size=12)
    ax.set_rlabel_position(0)
    plt.yticks([0.1, 0.2, 0.3, 0.4, 0.5], ["0.1", "0.2", "0.3", "0.4", "0.5"], color="grey", size=7)
    plt.ylim(0, max(means) + 0.1)

    # 4. 데이터 플롯
    ax.plot(angles, means, linewidth=2, linestyle='solid', color='blue')
    ax.fill(angles, means, 'b', alpha=0.1)

    plt.title(f"<{game_name}>\n평균 게임 성향 프로필 (Radar Chart)", size=20, y=1.1)

In [ ]:
# --- 4. 레이더 차트 저장 ---

if not df.empty:
    print("--- 4. 레이더 차트 저장 ---")
    
    # GAME_OUTPUT_DIR을 사용하여 저장 경로 설정
    radar_chart_filename = os.path.join(GAME_OUTPUT_DIR, f"{game_name}_성향_레이더차트.png")
    
    # 현재 활성화된 Matplotlib figure를 저장합니다.
    fig.savefig(radar_chart_filename, bbox_inches='tight')
    
    plt.close(fig) 

    print(f"✅ 레이더 차트 저장 완료: {radar_chart_filename}")
else:
    print("❌ 저장할 Figure 객체('fig')를 찾을 수 없거나 데이터프레임이 비어있습니다.")

In [ ]:
if not df.empty:
    plt.figure(figsize=(8, 6))
    
    # 성향 컬럼 간의 상관관계 계산
    corr = df[persona_cols].corr()
    
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1, linewidths=1)
    
    plt.title('성향 벡터 간 상관관계 분석', fontsize=15)

In [ ]:
# --- 5. 상관관계 히트맵 저장 ---

if not df.empty:
    print("--- 5. 상관관계 히트맵 생성 및 저장 ---")
    
    # GAME_OUTPUT_DIR을 사용하여 저장 경로 설정
    heatmap_filename = os.path.join(GAME_OUTPUT_DIR, f"{game_name}_성향_상관관계_히트맵.png")
    
    # 현재 활성화된 Matplotlib figure를 저장합니다.
    fig.savefig(heatmap_filename, bbox_inches='tight')
    
    plt.close(fig) 

    print(f"✅ 히트맵 저장 완료: {heatmap_filename}")
else:
    print("❌ 저장할 Figure 객체('fig')를 찾을 수 없거나 데이터프레임이 비어있습니다.")

In [ ]:
if not df.empty:
    plt.figure(figsize=(10, 6))
    
    # 0점인 데이터(성향 미분류) 제외
    plot_df = df[df[persona_cols].sum(axis=1) > 0]

    sns.boxplot(x='Dominant_Persona', y='playtime_hours', data=plot_df, palette="Set3")
    
    plt.title('주요 성향(Dominant Persona)별 플레이 시간 분포', fontsize=15)
    plt.xlabel('사용자의 주요 성향', fontsize=12)
    plt.ylabel('플레이 시간 (Hour)', fontsize=12)
    plt.xticks(rotation=0)
    plt.show()

In [ ]:
# --- 6. 성향별 플레이 시간 분포 저장 ---

if not df.empty:
    print("--- 6. 플레이 시간 분포 박스 플롯 생성 및 저장 ---")
    
    # GAME_OUTPUT_DIR을 사용하여 저장 경로 설정
    boxplot_filename = os.path.join(GAME_OUTPUT_DIR, f"{game_name}_플레이시간_분포.png")
    
    # 현재 활성화된 Matplotlib figure를 저장합니다.
    fig.savefig(boxplot_filename, bbox_inches='tight')
    
    plt.close(fig) 

    print(f"✅ 박스 플롯 저장 완료: {boxplot_filename}")
else:
    print("❌ 저장할 Figure 객체('fig')를 찾을 수 없거나 데이터프레임이 비어있습니다.")

In [ ]:
if not df.empty:
    # 모든 리뷰 텍스트 합치기
    text_all = " ".join(df['review_text'].astype(str).tolist())

    # 워드클라우드 생성 (한글 폰트 경로 필수)
    wc = WordCloud(
        font_path=font_path, # 폰트 경로 (Windows/Mac 자동 설정됨)
        background_color='white',
        width=800,
        height=400,
        colormap='viridis',
        stopwords=['게임', '진짜', '너무', '정말', '하고', '있는', '합니다', '그냥'] # 불용어 추가 가능
    ).generate(text_all)

    plt.figure(figsize=(12, 6))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"<{game_name}>\n리뷰 주요 키워드 워드클라우드", fontsize=15)
    plt.show()

In [ ]:
wordcloud_filename = os.path.join(GAME_OUTPUT_DIR, f"{game_name}_워드클라우드.png")
wc.to_file(wordcloud_filename) 
print(f"✅ 워드 클라우드 저장 완료: {wordcloud_filename}")

In [ ]:
# 성향 컬럼들만 선택하여 평균 계산
if not df.empty:
    persona_columns = [col for col in df.columns if col.startswith('S_')]
    average_persona = df[persona_columns].mean().sort_values(ascending=False)
    
    fig = plt.figure(figsize=(8, 5))

    sns.barplot(
        x=average_persona.index, 
        y=average_persona.values, 
        hue=average_persona.index, # x 변수를 hue에 할당
        palette="viridis",
        legend=False # 범례(legend)는 필요 없으므로 False로 설정
    )

    plt.title(f'{game_name} 리뷰어들의 평균 성향 벡터', fontsize=15)
    plt.ylabel('평균 가중치', fontsize=12)
    plt.xticks(rotation=0)

In [ ]:
# --- 7. 평균 성향 벡터 저장 ---

if not df.empty:
    print("--- 7. 플레이어 평균 성향 벡터 생성 및 저장 ---")
    
    # GAME_OUTPUT_DIR을 사용하여 저장 경로 설정
    borplot_filename = os.path.join(GAME_OUTPUT_DIR, f"{game_name}_성향_막대그래프_분포.png")
    
    # 현재 활성화된 Matplotlib figure를 저장합니다.
    fig.savefig(borplot_filename, bbox_inches='tight')
    
    plt.close(fig) 

    print(f"✅ 박스 플롯 저장 완료: {borplot_filename}")
else:
    print("❌ 저장할 Figure 객체('fig')를 찾을 수 없거나 데이터프레임이 비어있습니다.")

In [ ]:
# 플레이 시간을 시간 단위로 변환
if not df.empty:
    df['playtime_hours'] = df['playtime_forever'] / 60
    
    # 플레이 시간에 따른 분위기 파악
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='voted_up', y='playtime_hours', data=df, showfliers=False) # 이상치 제외
    plt.title(f'{game_name} 추천/비추천별 플레이 시간 분포', fontsize=15)
    plt.xlabel('추천 여부 (False: 비추천, True: 추천)', fontsize=12)
    plt.ylabel('플레이 시간 (시간)', fontsize=12)
    plt.show()

In [ ]:
# --- 8. 추천 비율별 플레이 시간 저장 ---

if not df.empty:
    print("--- 8. 추천 비율별 플레이 시간 생성 및 저장 ---")
    
    # GAME_OUTPUT_DIR을 사용하여 저장 경로 설정
    timeboxplot_filename = os.path.join(GAME_OUTPUT_DIR, f"{game_name}_추천비율_분포.png")
    
    # 현재 활성화된 Matplotlib figure를 저장합니다.
    fig.savefig(timeboxplot_filename, bbox_inches='tight')
    
    plt.close(fig) 

    print(f"✅ 박스 플롯 저장 완료: {timeboxplot_filename}")
else:
    print("❌ 저장할 Figure 객체('fig')를 찾을 수 없거나 데이터프레임이 비어있습니다.")

In [ ]:
if not df.empty:
    # 분석 결과 파일 경로 생성
    output_csv_filename = f"analyzed_{game_name}_reviews.csv"
    output_full_path = os.path.join(GAME_OUTPUT_DIR, output_csv_filename)
    absolute_full_path = os.path.abspath(output_full_path)
    df.to_csv(absolute_full_path, index=False, encoding='utf-8')
    print(f"\n📂 분석 및 시각화 완료. 결과 저장됨: {absolute_full_path}")